the only exception is that the potential can no longer be calculated explicitly beforehand. You have to pay attention to the `storage` argument of the potential, which determines where the precalculated potential is stored. The options are

* `device` : (default) The potential is stored on the GPU
* `host` : The potential is in the host memory and individual slices are transferred to the GPU
* `disk` : The potential is saved to disk and read back in chunks

The distinction between `device` and `host` only matters, if you are doing GPU calculations. `device` is always the preferred option, but may not be possible due to memory constraints. Choosing disk should only be done as a workaround, for systems with very low memory.